In [47]:
import os
import glob
import pandas
import tqdm
import numpy as np
from PIL import Image

import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [48]:
dir_path="/media/syz/Remote_Sensing/dataset/potsdam/IRRG"

In [49]:
class MyDataset(Dataset):
    def __init__(self, dataset_path):
#         self.data = torch.randn(100, 3, 24, 24)
        self.datalist = glob.glob(os.path.join(dataset_path, '*.tif'))

    def __getitem__(self, index):
        
        fp = self.datalist[index]
        x = Image.open(fp)
        arrayx = np.array(x)
        torchx = torch.from_numpy(arrayx)
        
        return torchx.float()

    def __len__(self):
        return len(self.datalist)

In [50]:
# refer to: https://discuss.pytorch.org/t/about-normalization-using-pre-trained-vgg16-networks/23560/20
dataset = MyDataset(dataset_path=dir_path)
loader = DataLoader(
    dataset,
    batch_size=1,
    num_workers=1,
    shuffle=False
)

nb_samples = 0.
channel_mean = torch.Tensor([0., 0., 0.])
channel_std = torch.Tensor([0., 0., 0.])
for images in tqdm.tqdm(loader):
    # scale image to be between 0 and 1 
    images=images/255.
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1)*images.size(2), 3)
    for i in range(3):
        channel_mean[i]+=images[:, :,i].mean(1).sum(0)
        channel_std[i]+=images[:, :,i].std(1).sum(0)
    nb_samples += batch_samples

channel_mean /= nb_samples
channel_std /= nb_samples
 
print(f"channel mean: {channel_mean}, channel_std: {channel_std}")

100%|██████████| 38/38 [00:37<00:00,  1.02it/s]channel mean: tensor([0.3829, 0.3629, 0.3369]), channel_std: tensor([0.1388, 0.1377, 0.1431])

